# 세션 5 – 다중 에이전트 오케스트레이터

Foundry Local을 사용하여 간단한 두 에이전트 파이프라인(Researcher -> Editor)을 보여줍니다.


### 설명: 종속성 설치
로컬 모델 액세스 및 채팅 완료에 필요한 `foundry-local-sdk`와 `openai`를 설치합니다. 멱등성을 가집니다.


# 시나리오
최소한의 두 에이전트 오케스트레이터 패턴을 구현:
- **Researcher 에이전트**는 간결한 사실 중심의 요점을 수집
- **Editor 에이전트**는 이를 경영진이 이해하기 쉽게 재작성

에이전트별 공유 메모리, 중간 출력의 순차적 전달, 간단한 파이프라인 기능을 보여줍니다. 더 많은 역할(예: Critic, Verifier) 또는 병렬 분기로 확장 가능.

**환경 변수:**
- `FOUNDRY_LOCAL_ALIAS` - 기본적으로 사용할 모델 (기본값: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - 주요 에이전트 모델 (ALIAS를 재정의)
- `AGENT_MODEL_EDITOR` - Editor 에이전트 모델 (기본값: 주요 모델)

**SDK 참조:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**작동 방식:**
1. **FoundryLocalManager**가 Foundry Local 서비스를 자동으로 시작
2. 지정된 모델을 다운로드 및 로드 (또는 캐시된 버전 사용)
3. 상호작용을 위한 OpenAI 호환 엔드포인트 제공
4. 각 에이전트는 특화된 작업을 위해 다른 모델을 사용할 수 있음
5. 내장된 재시도 로직으로 일시적인 오류를 원활하게 처리

**주요 기능:**
- ✅ 자동 서비스 검색 및 초기화
- ✅ 모델 라이프사이클 관리 (다운로드, 캐시, 로드)
- ✅ 친숙한 API를 위한 OpenAI SDK 호환성
- ✅ 에이전트 특화 작업을 위한 다중 모델 지원
- ✅ 재시도 로직을 통한 강력한 오류 처리
- ✅ 로컬 추론 (클라우드 API 불필요)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### 설명: 핵심 임포트 및 타입 지정
에이전트 메시지 저장을 위한 데이터 클래스와 명확성을 위한 타입 힌트를 소개합니다. 이후 에이전트 작업을 위해 Foundry Local 매니저와 OpenAI 클라이언트를 임포트합니다.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### 설명: 모델 초기화 (SDK 패턴)
Foundry Local Python SDK를 사용하여 강력한 모델 관리를 제공합니다:
- **FoundryLocalManager(alias)** - 서비스를 자동으로 시작하고 별칭으로 모델을 로드합니다.
- **get_model_info(alias)** - 별칭을 구체적인 모델 ID로 변환합니다.
- **manager.endpoint** - OpenAI 클라이언트를 위한 서비스 엔드포인트를 제공합니다.
- **manager.api_key** - API 키를 제공합니다 (로컬 사용 시 선택 사항).
- 서로 다른 에이전트(기본 에이전트 vs 편집 에이전트)를 위한 개별 모델을 지원합니다.
- 내장된 지수적 백오프를 활용한 재시도 로직으로 복원력을 제공합니다.
- 서비스 준비 상태를 확인하기 위한 연결 검증 기능을 포함합니다.

**주요 SDK 패턴:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**수명 주기 관리:**
- 매니저는 전역적으로 저장되어 적절한 정리가 가능합니다.
- 각 에이전트는 전문화를 위해 다른 모델을 사용할 수 있습니다.
- 자동 서비스 검색 및 연결 처리 기능을 제공합니다.
- 실패 시 지수적 백오프를 활용한 우아한 재시도 기능을 포함합니다.

이를 통해 에이전트 오케스트레이션이 시작되기 전에 적절한 초기화를 보장합니다.

**참고:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### 설명: Agent 및 Memory 클래스
`AgentMsg`라는 가벼운 메모리 항목과 `Agent`를 정의하여 다음을 캡슐화합니다:
- **시스템 역할** - 에이전트의 페르소나와 지침
- **메시지 기록** - 대화 컨텍스트 유지
- **act() 메서드** - 적절한 오류 처리를 통해 작업 실행

에이전트는 서로 다른 모델(기본 모델 vs 편집 모델)을 사용할 수 있으며, 에이전트별로 독립된 컨텍스트를 유지합니다. 이 패턴은 다음을 가능하게 합니다:
- 작업 간 메모리 지속성
- 에이전트별 유연한 모델 할당
- 오류 격리 및 복구
- 쉬운 체이닝 및 오케스트레이션


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### 설명: 오케스트레이션된 파이프라인
두 개의 전문화된 에이전트를 생성합니다:
- **Researcher**: 주요 모델을 사용하여 사실 정보를 수집
- **Editor**: 별도의 모델을 사용할 수 있으며(구성된 경우), 정보를 다듬고 재작성

`pipeline` 함수:
1. Researcher가 원시 정보를 수집
2. Editor가 이를 실행 가능한 최종 결과물로 다듬음
3. 중간 결과와 최종 결과를 반환

이 패턴은 다음을 가능하게 합니다:
- 모델 전문화 (역할에 따라 다른 모델 사용)
- 다단계 처리를 통한 품질 향상
- 정보 변환 과정의 추적 가능성
- 더 많은 에이전트 추가 또는 병렬 처리로의 확장 용이성


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### 설명: 파이프라인 실행 및 결과
컴플라이언스 + 지연 시간 주제의 질문에 대해 다중 에이전트 파이프라인을 실행하여 다음을 시연합니다:
- 다단계 정보 변환
- 에이전트의 전문화와 협업
- 정제를 통한 출력 품질 향상
- 추적 가능성 (중간 및 최종 출력 모두 보존)

**결과 구조:**
- `question` - 원래 사용자의 질문
- `research` - 원시 연구 결과 (사실 기반의 요약)
- `final` - 정제된 요약본
- `models` - 각 단계에서 사용된 모델

**확장 아이디어:**
1. 품질 검토를 위한 Critic 에이전트 추가
2. 다양한 측면에 대한 병렬 연구 에이전트 구현
3. 사실 확인을 위한 Verifier 에이전트 추가
4. 복잡성 수준에 따라 다른 모델 사용
5. 반복적 개선을 위한 피드백 루프 구현


### 고급: 사용자 지정 에이전트 구성

초기화 셀을 실행하기 전에 환경 변수를 수정하여 에이전트 동작을 사용자 지정해 보세요:

**사용 가능한 모델:**
- 터미널에서 `foundry model ls`를 사용하여 모든 사용 가능한 모델을 확인하세요
- 예시: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b 등


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
